# Experimentos

**¿Qué representacones deberíamos a comparar?**
<ul>
<li>Bag of words (sin balancear, con todos los algoritmos "clásicos")</li>
<li>Tf-idf model (sin balancear, con todos los algoritmos "clásicos"</li>

-----

<li>Documento representado como Average of word embeddings usando modelo w2v entrenado sobre EMR (con todos los algoritmos clásicos y sin balancear)</li>

<li>Documento representado como Average of word embeddings usando modelo w2v entrenado sobre Spanish wikipedia (http://crscardellino.me/SBWCE/, https://github.com/Kyubyong/wordvectors). Tenemos que mirar si esos modelos preentrenados hacen stemming o no. Seguramente no lo hagan, y por tanto, nosotros no vamos a hacer stemming (con todos los algoritmos clásicos y sin balancear)</li>

-----

<li>bag of centroids (sobre el modelo de EMRs)</li>
<li>bag of centroids (sobre el modelo de wikipedia)</li>
</ul>

Mi idea es quitar ElasticSearch y dejar un bseline de bolsa de palabras 

**¿Qué métodos (clasificadores) deberíamos a comparar?**

<ul>
<li>Classifiers: **SVM, MLP, RandomForest, MultinomialNB**, KNeighborsClassifier, AdaBoostClassifier, GaussianNB, QuadraticDiscriminantAnalysis</li>

<li>Deberíamos encontrar la mejor setting para cada uno de ellos? GridSearchCV </li>

<li>Ensemble (Voting): SVM, MLP, RandomForest, MultinomialNB, KNeighborsClassifier, AdaBoostClassifier, GaussianNB, QuadraticDiscriminantAnalysis</li>
<li>CNN: malos resultados</li>
</ul>



In [1]:
import pandas
import csv
import re
import unicodedata
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem import SnowballStemmer
import time
import scipy.sparse
import warnings

#classifiers
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

#graphs
%matplotlib inline
import matplotlib.pyplot as plt

#parallel
from sklearn.externals.joblib import Parallel, delayed

## Carga de datos en memoria
En la siguiente sección cargamos los datos de entrenamiento y de test

In [2]:
df_train = pandas.read_csv('./data/train', sep='\t', index_col=0)
df_test = pandas.read_csv('./data/test', sep='\t', index_col=0)

Xtrain=df_train['text']
Ytrain=df_train['label']

Xtest=df_test['text']
Ytest=df_test['label']
print('datasets were loaded', len(df_train), len(df_test))

datasets were loaded 164926 54976


A continuación definimos "split_into_tokens", función encargada de procesar el texto dando como resultado una lista de tokens donde se han relizado los siguientes pasos:
<li> Los signos de acentuación son eliminados
<li> Los caracteres no alfanuméricos son filtrados
<li> Mayus a minus y split del texto en tokens
<li> Eliminadas las stopwords y sustitución de las palabras restantes por su raiz (stemming)

In [9]:
def split_into_tokens(text):
    #stemmer = SnowballStemmer('spanish')
    stemmer = None
    min_length = 3
    # 1. Remove accent marks
    review_text = ''.join((c for c in unicodedata.normalize('NFD',str(text)) if unicodedata.category(c) != 'Mn'))
    #
    # 2. Remove non-alphanumeric
    #letters_only = re.sub("[^A-Za-z0-9]", " ", review_text) 
    letters_only = re.sub("[^\w\d]", " ", review_text) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("spanish"))                  
    # 
    # 4. Remove stop words and apply or not stemming
    if stemmer:
        filtered_tokens = [stemmer.stem(w) for w in words if not w in stops and len(w)>=min_length]
    else:
        filtered_tokens = [w for w in words if not w in stops and len(w)>=min_length]
    #
    # 5. return the result
    return filtered_tokens

In [11]:
prueba = "Refiere intermitente sensacion de acorchamiento en zona frontal derecha y hemicara derecha"
prueba2 = "No disminucion de la diuresis. No claudicacion mandibular. No amaurosis"

filtrado = split_into_tokens(prueba2)
filtrado

['disminucion', 'diuresis', 'claudicacion', 'mandibular', 'amaurosis']

In [ ]:
bow = CountVectorizer(analyzer=split_into_tokens)

print("Creando matriz de bolsa de palabras...")

bow.fit(Xtrain, Ytrain)
Xprueba_bow = bow.transform(Xtrain)

Xprueba_bow

La función "trainModel" recibe el nombre del algoritmo de clasificación, la clase, sus parametros y los conjuntos de datos. Dicha función se encarga de entrenar el modelo y devolver una tupla con el nombre del algoritmo y el modelo ya entrenado.

In [10]:
def trainModel(name, clazz, params, Xtrain, Ytrain):
    print("training ", name)
    model = clazz(**params)
    start = time.time() # Start time
    model.fit(Xtrain, Ytrain)
    end = time.time()
    elapsed = end - start
    print("-> done ", name, " - Time taken for training:", elapsed, "seconds")
    return (name, model)

## Algoritmos "clásicos"

En esta sección creamos un diccionario que contiene todos los datos necesarios de los algoritmos de clasificación que se van a emplear, así como los parametros de cada uno de ellos. (En caso de no estar indicados los parametros, se emplean aquellos configurados por defecto en scikit-learn)

In [11]:
estimators = {"KNeighbors": (KNeighborsClassifier, {}),
              "MultinomialNB" : (MultinomialNB, {}),
              "RandomForest" : (RandomForestClassifier, {"n_estimators":100}),
              "LogisticRegression" : (LogisticRegression, {}),
              "MLP" : (MLPClassifier, {"hidden_layer_sizes":100}),
              "SVM" : (SVC, {"cache_size":1000}),
              "LinearSVC" : (LinearSVC, {})
             }

## Bag of words (sin balancear, con todos los algoritmos "clásicos")


En esta sección se transforman los datos de entrenamiento y de test en bolsa de palabras, pasando de un conjunto de tokens, a un conjunto de número de ocurrencias por cada token.

In [4]:
bow = CountVectorizer(analyzer=split_into_tokens)

print("Creando matriz de bolsa de palabras...")

%time bow.fit(Xtrain, Ytrain)
%time Xtrain_bow = bow.transform(Xtrain)
%time Xtest_bow = bow.transform(Xtest)

scipy.sparse.save_npz('data/Xtrain_bow.npz', Xtrain_bow)
scipy.sparse.save_npz('data/Xtest_bow.npz', Xtest_bow)

Creando matriz de bolsa de palabras...
CPU times: user 8min 42s, sys: 1.4 s, total: 8min 43s
Wall time: 8min 43s
CPU times: user 8min 43s, sys: 1.25 s, total: 8min 44s
Wall time: 8min 44s
CPU times: user 2min 53s, sys: 448 ms, total: 2min 53s
Wall time: 2min 53s


In [6]:
Xtest_bow.shape

(54976, 140425)

Esta sección carga en memoria las bolsas de palabras. Emplear solo en el caso de haber obtenido previamente las bolsas de palabras y no tenerlas cargadas en memoria.

In [12]:
# Xtest_bow = scipy.sparse.load_npz('data/Xtest_bow.npz').astype(np.int16, casting='same_kind')
# Xtrain_bow = scipy.sparse.load_npz('data/Xtrain_bow.npz').astype(np.int16, casting='same_kind')

A continuación, creamos tantos procesos como estimadores tengamos. Dichos procesos van a estar encargados de entrenar, cada uno y de forma paralela, un modelo de clasificación distinto.<br> 
Como resultado obtenemos una lista de tuplas formadas por: (el nombre del algoritmo, modelo ya entrenado).

In [7]:
models = Parallel(n_jobs=len(estimators))(delayed(trainModel)(name, clazz, params, Xtrain_bow, Ytrain) for (name, (clazz, params)) in estimators.items())

training  KNeighbors
-> done  KNeighbors  - Time taken for training: 0.05667233467102051 seconds
training  MultinomialNB
-> done  MultinomialNB  - Time taken for training: 0.1639249324798584 seconds
training  MLP
training  RandomForest
training  LogisticRegression
training  LinearSVC
training  SVM
-> done  LinearSVC  - Time taken for training: 2.9192557334899902 seconds
-> done  LogisticRegression  - Time taken for training: 17.37110424041748 seconds
-> done  RandomForest  - Time taken for training: 52.50565981864929 seconds
-> done  SVM  - Time taken for training: 562.716991186142 seconds
-> done  MLP  - Time taken for training: 2464.838773727417 seconds


Una vez entrenados los modelos, obtenemos la clase de los individuos del conjunto de test y evaluamos los resultados de cada uno de ellos. 

In [8]:
print("Obteniendo resultados:")
for (name, model) in models:
    start = time.time() # Start time
    if name == "KNeighbors":
        result = [y for x in [Xtest_bow[i:i+5000,:] for i in range(0,Xtest_bow.shape[0],5000)] for y in model.predict(x)]
    else:
        result = model.predict(Xtest_bow)
    end = time.time()
    elapsed = end - start
    print("---------- Modelo: ", name, " ---------- Time taken for prediction:", elapsed,"seconds\n", classification_report(Ytest, result, digits=4), "\n")

Obteniendo resultados:
---------- Modelo:  KNeighbors  ---------- Time taken for prediction: 661.5408456325531 seconds
              precision    recall  f1-score   support

      False     0.9987    1.0000    0.9994     54473
       True     1.0000    0.8628    0.9264       503

avg / total     0.9987    0.9987    0.9987     54976
 

---------- Modelo:  MultinomialNB  ---------- Time taken for prediction: 0.11231493949890137 seconds
              precision    recall  f1-score   support

      False     0.9992    0.9976    0.9984     54473
       True     0.7761    0.9165    0.8405       503

avg / total     0.9972    0.9968    0.9969     54976
 

---------- Modelo:  MLP  ---------- Time taken for prediction: 0.4970865249633789 seconds
              precision    recall  f1-score   support

      False     0.9994    0.9996    0.9995     54473
       True     0.9551    0.9304    0.9426       503

avg / total     0.9990    0.9990    0.9990     54976
 

---------- Modelo:  RandomForest  --

## Tf-idf model (sin balancear, con todos los algoritmos "clásicos")

Para comparar con los resultados obtenidos, emplearemos la representación de los datos de entrenamiento y test en formato de Tf-idf. Para ello, la bolsa de palabras es transformada en la frecuencia de ocurrencia de los términos en la colección de documentos.<br>
El proceso que se lleva a cabo, a partir de este punto, es el mismo que el empleado con la bolsa de palabras anteriormente.

In [7]:
tfidf = TfidfTransformer()

print("Creando matriz de tf-idf...")

%time tfidf.fit(Xtrain_bow, Ytrain)
%time Xtrain_tfidf = tfidf.transform(Xtrain_bow)
%time Xtest_tfidf = tfidf.transform(Xtest_bow)

scipy.sparse.save_npz('data/Xtrain_tfidf.npz', Xtrain_tfidf)
scipy.sparse.save_npz('data/Xtest_tfidf.npz', Xtest_tfidf)

Creando matriz de tf-idf...
CPU times: user 64 ms, sys: 24 ms, total: 88 ms
Wall time: 85.9 ms
CPU times: user 412 ms, sys: 140 ms, total: 552 ms
Wall time: 549 ms
CPU times: user 144 ms, sys: 32 ms, total: 176 ms
Wall time: 175 ms


Esta sección carga en memoria la representación de los datos en tf-idf. Emplear solo en el caso de haber obtenido previamente dichos datos y no tenerlos cargadas en memoria.

In [16]:
# Xtest_tfidf = scipy.sparse.load_npz('data/Xtest_tfidf.npz').astype(np.float32)
# Xtrain_tfidf = scipy.sparse.load_npz('data/Xtrain_tfidf.npz').astype(np.float32)

In [17]:
Xtrain_tfidf

<164926x140425 sparse matrix of type '<class 'numpy.float32'>'
	with 18552095 stored elements in Compressed Sparse Row format>

A continuación, creamos tantos procesos como estimadores tengamos. Dichos procesos van a estar encargados de entrenar, cada uno y de forma paralela, un modelo de clasificación distinto.<br> 
Como resultado obtenemos una lista de tuplas formadas por: (el nombre del algoritmo, modelo ya entrenado).

In [10]:
models_tfidf = Parallel(n_jobs=len(estimators))(delayed(trainModel)(name, clazz, params, Xtrain_tfidf, Ytrain) for (name, (clazz, params)) in estimators.items())

training  KNeighbors
training  MultinomialNB
-> done  KNeighbors  - Time taken for training: 0.29846620559692383 seconds
-> done  MultinomialNB  - Time taken for training: 0.17311930656433105 seconds
training  MLP
training  RandomForest
training  LogisticRegression
training  LinearSVC
training  SVM
-> done  LinearSVC  - Time taken for training: 1.6419150829315186 seconds
-> done  LogisticRegression  - Time taken for training: 5.75917911529541 seconds
-> done  RandomForest  - Time taken for training: 58.53112745285034 seconds
-> done  SVM  - Time taken for training: 429.11979126930237 seconds
-> done  MLP  - Time taken for training: 4001.9736495018005 seconds


Una vez entrenados los modelos, obtenemos la clase de los individuos del conjunto de test y evaluamos los resultados de cada uno de ellos. 

In [11]:
print("Obteniendo resultados:")
for (name, model) in models_tfidf:
    start = time.time() # Start time
    if name == "KNeighbors":
        result = [y for x in [Xtest_tfidf[i:i+5000,:] for i in range(0,Xtest_tfidf.shape[0],5000)] for y in model.predict(x)]
    else:
        result = model.predict(Xtest_tfidf)
    end = time.time()
    elapsed = end - start
    print("---------- Modelo: ", name, " ---------- Time taken for prediction:", elapsed,"seconds\n", classification_report(Ytest, result, digits=4), "\n")

Obteniendo resultados:
---------- Modelo:  KNeighbors  ---------- Time taken for prediction: 632.7096681594849 seconds
              precision    recall  f1-score   support

      False     0.9977    1.0000    0.9988     54473
       True     1.0000    0.7475    0.8555       503

avg / total     0.9977    0.9977    0.9975     54976
 

---------- Modelo:  MultinomialNB  ---------- Time taken for prediction: 0.03978276252746582 seconds
              precision    recall  f1-score   support

      False     0.9922    1.0000    0.9961     54473
       True     1.0000    0.1451    0.2535       503

avg / total     0.9922    0.9922    0.9893     54976
 

---------- Modelo:  MLP  ---------- Time taken for prediction: 0.5144734382629395 seconds
              precision    recall  f1-score   support

      False     0.9994    0.9997    0.9995     54473
       True     0.9670    0.9324    0.9494       503

avg / total     0.9991    0.9991    0.9991     54976
 

---------- Modelo:  RandomForest  --

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Voting (Bag of Words)

In [12]:
ensemble = VotingClassifier(models, n_jobs=-1)
start = time.time() # Start time
voting_model_bow=ensemble.fit(Xtrain_bow,Ytrain)
end = time.time()
elapsed = end - start
print("VotingClassifier - Time taken for training:", elapsed, "seconds")

VotingClassifier - Time taken for training: 2449.9408764839172 seconds


In [13]:
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
start = time.time() # Start time
predictions1 = [y for x in [Xtest_bow[i:i+2000,:] for i in range(0,Xtest_bow.shape[0],2000)] for y in voting_model_bow.predict(x)]
end = time.time()
elapsed = end - start
print("VotingClassifier - Time taken for prediction:", elapsed, "seconds")
cr1=classification_report(Ytest, predictions1, digits=4)
print(cr1)

VotingClassifier - Time taken for prediction: 800.65008020401 seconds
             precision    recall  f1-score   support

      False     0.9992    0.9999    0.9996     54473
       True     0.9935    0.9085    0.9491       503

avg / total     0.9991    0.9991    0.9991     54976



## Voting (Tf-idf)

In [14]:
ensemble = VotingClassifier(models_tfidf, n_jobs=-1)
start = time.time() # Start time
voting_model_tfidf=ensemble.fit(Xtrain_tfidf,Ytrain)
end = time.time()
elapsed = end - start
print("VotingClassifier - Time taken for training:", elapsed, "seconds")

VotingClassifier - Time taken for training: 3767.6675279140472 seconds


In [15]:
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
start = time.time() # Start time
predictions2 = [y for x in [Xtest_tfidf[i:i+2000,:] for i in range(0,Xtest_tfidf.shape[0],2000)] for y in voting_model_tfidf.predict(x)]
end = time.time()
elapsed = end - start
print("VotingClassifier - Time taken for prediction:", elapsed, "seconds")
cr2=classification_report(Ytest, predictions2, digits=4)
print(cr2)

VotingClassifier - Time taken for prediction: 731.4793696403503 seconds
             precision    recall  f1-score   support

      False     0.9988    1.0000    0.9994     54473
       True     1.0000    0.8728    0.9321       503

avg / total     0.9988    0.9988    0.9988     54976

